In [1]:
import decimal

decimal.getcontext()
decimal.getcontext().prec = 14

In [5]:
# Context manager for custom precision during a particular calculation
class precision:
    def __init__(self, prec):
        self.prec = prec
        self.current_prec = decimal.getcontext().prec

    def __enter__(self):
        decimal.getcontext().prec = self.prec

    def __exit__(self, exc_type, exc_value, exc_tb):
        decimal.getcontext().prec = self.current_prec
        return False
    
with precision(3):
    print(decimal.Decimal(1) / decimal.Decimal(3))

print(decimal.Decimal(1) / decimal.Decimal(3))
    

0.333
0.33333333333333


In [7]:
# the decimal module has incorportated context manager
with decimal.localcontext() as ctx:
    ctx.prec = 3
    print(decimal.Decimal(1) / decimal.Decimal(3))

print(decimal.Decimal(1) / decimal.Decimal(3))

0.333
0.33333333333333


In [9]:
# Context manager example to count the execution time
from time import perf_counter, sleep

class Timer:
    def __init__(self):
        self.elapsed = 0

    def __enter__(self):
        self.start = perf_counter()
        return self
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        self.stop = perf_counter()
        self.elapsed = self.stop - self.start
        return False
    
with Timer()as timer:
    sleep(1)

    print(timer.elapsed)

0


In [15]:
# Context manager to print output to a file rather than to the console

import sys

class OutToFile:
    def __init__(self, fname):
        self._fname = fname
        self._current_stdout = sys.stdout

    def __enter__(self):
        self._file = open(self._fname, 'w')
        sys.stdout = self._file

    def __exit__(self, exc_type, exc_value, exc_tb):
        sys.stdout = self._current_stdout
        self._file.close()
        return False
    

with OutToFile('test.txt'):
    print(sys.stdout)
    print('line 1')
    print('line 2')

with open('test.txt') as f:
    print(f.readlines())

["<_io.TextIOWrapper name='test.txt' mode='w' encoding='cp1252'>\n", 'line 1\n', 'line 2\n']


In [19]:
# Context manager to add tags to text

class Tag:
    def __init__(self, tag):
        self._tag = tag

    def __enter__(self):
        print(f'<{self._tag}>', end = '')

    def __exit__(self, exc_type, exc_value, exc_tb):
        print(f'</{self._tag}>', end = '')
        return False
    

with Tag('p'):
    print('some ', end = '')
    with Tag('b'):
        print('bold', end = '')
    print(' text', end = '')
    

<p>some <b>bold</b> text</p>

In [24]:
# Context manager to create a list with proper indentation
class ListMaker:
    def __init__(self, title, prefix='- ', indent=3):
        self._title = title
        self._prefix = prefix
        self._indent = indent
        self._current_indent = 0
        print(title)

    def __enter__(self):
        self._current_indent += self._indent
        return self
    
    def __exit__(self, exc_type, exc_value, exc_tb):
        self._current_indent -= self._indent

    def print_item(self, arg):
        s = ' ' * self._current_indent + self._prefix + str(arg)
        print(s)


with ListMaker('Items') as lm:
    lm.print_item('Item 1')
    with lm:
        lm.print_item('sub item 1a')
        lm.print_item('sub item 1b')
        with lm:
            lm.print_item('sub sub item 1b1')
        lm.print_item('sub item 1c')
    lm.print_item('Item 2')

Items
   - Item 1
      - sub item 1a
      - sub item 1b
         - sub sub item 1b1
      - sub item 1c
   - Item 2


In [26]:
# Combining multilple context managers
with OutToFile('my_list.txt'):
    
    with ListMaker('Items') as lm:
        lm.print_item('Item 1')
        with lm:
            lm.print_item('sub item 1a')
            lm.print_item('sub item 1b')
            with lm:
                lm.print_item('sub sub item 1b1')
            lm.print_item('sub item 1c')
        lm.print_item('Item 2') 